In [4]:
import gymnasium as gym
import tensorflow as tf
from tensorflow import keras

In [6]:
env = gym.make('CartPole-v1')

Action Space is a [Discrete](https://gymnasium.farama.org/api/spaces/fundamental/#gymnasium.spaces.Discrete) type
A space consisting of finitely many elements.
In CartPole-v1, it has two actions: left or right

In [17]:
print(type(env.action_space))
print(f"Action space size: {env.action_space.n}")
print(f"Action space start value: {env.action_space.start}")
env.action_space

<class 'gymnasium.spaces.discrete.Discrete'>
Action space size: 2
Action space start value: 0


Discrete(2)

Observation space is a 4 dimension state: 
- Cart Position
- Cart Velocity
- Pole Angle
- Pole Velocity at tip

In [20]:
print(type(env.observation_space))
# print(f"Action space size: {env.observation_space.n}")
# print(f"Action space start value: {env.observation_space.start}")
env.observation_space

<class 'gymnasium.spaces.box.Box'>


Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)